In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import psycopg2

In [4]:
from config import postgresql_home 
with psycopg2.connect(
            host = postgresql_home['host'],
            user = postgresql_home['user'], 
            password = postgresql_home['password'],
            database = 'competition_analysis'
            ) as connection:
    sales = pd.read_sql('SELECT * FROM sales', connection)

C:\Users\Боря\AppData\Local\Temp\ipykernel_8820\3356721468.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales = pd.read_sql('SELECT * FROM sales', connection)


In [5]:
sales.head()

,Дата,Артель,Плата
0,2048-01-01,Каша из топора,10208.64
1,2048-01-01,Бабки Ёжки,21961.11
2,2048-01-01,Хоббитонские угощения,6905.58
3,2048-01-01,Дырявый котёл,4571.70
4,2048-01-01,Ёлки-палки,15124.69


In [13]:
cube = pd.read_csv('cube.csv')
cube.head()

,Чеболь,Промысел,Царство,Плата
0,Дуремар,артефакторика,Лукоморье,54196.00
1,Дуремар,артефакторика,Нарния,126804.89
2,Дуремар,артефакторика,Тридесятое,110733.37
3,Дуремар,всячина,Лукоморье,74514.85
4,Дуремар,всячина,Нарния,24891.29


In [21]:
sql_create_query = '''
    CREATE TABLE IF NOT EXISTS cube (
        Чеболь       VARCHAR(64), 
        Промысел     VARCHAR(64), 
        Царство      VARCHAR(64), 
        Сумма_продаж DECIMAL(10, 2)
    )
'''

In [27]:
with psycopg2.connect(
            host = postgresql_home['host'],
            user = postgresql_home['user'], 
            password = postgresql_home['password'],
            database = 'competition_analysis'
            ) as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql_create_query)

In [31]:
## надо, видимо, использовать SQLAlchemy https://sql-ex.ru/blogs/?/Samye_bystrye_metody_massovoj_vstavki_frejma_dannyh_Pandas_v_PostgreSQL.html
with psycopg2.connect(
            host = postgresql_home['host'],
            user = postgresql_home['user'], 
            password = postgresql_home['password'],
            database = 'competition_analysis'
            ) as connection:
    cube.to_sql(
        name='cube', # имя таблицы
        con=connection,  # движок
        if_exists="append", # если таблица уже существует, добавляем
        index=False # без индекса
    )

C:\Users\Боря\AppData\Local\Temp\ipykernel_8820\2284319929.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cube.to_sql(


DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': syntax error at or near ";"
LINE 8:             AND name=?;
                              ^
